In [23]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


 
# train word2vec on the two sentences
model = gensim.models.Word2Vec.load('./corpus/word2vec300_phrase.model')

In [22]:
phrase = gensim.models.phrases.Phraser.load('corpus/phrase.model')

In [24]:
competency = {'creativity': 'ability to think in a divergent way, and generate novel ideas.',
 'employee loyalty': 'be committed to the success of the organization and believe that working for this organization is their best option.',
 'leadership': 'ability to engage and facilitate others to create a better result.',
 'passion': 'desire to provide long-term commitment to their organization, demonstrate peak performance, and maintain increased tenure with the organization. with the intense desire or enthusiasm for the work.',
 'people skills': 'ability to establish social connections and understand the mindset of others.',
 'proactivity': 'take the initiative and responsibility in improving business, rather than looking for causes in outside circumstances or other people.',
 'reliability': 'ability to performing to perform when needed, finishing projects and meeting deadlines.',
 'teamwork': 'ability to adapt to the needs of a group of people, while supporting the work of others.'}

In [56]:
comps = ['_'.join(c.split()) + ' ' + competency[c] for c in competency]
comps = [c.replace('.', '') for c in comps]
comps = [c.replace(',', '') for c in comps]
comps = [c.split() for c in comps] 
comps

[['leadership',
  'ability',
  'to',
  'engage',
  'and',
  'facilitate',
  'others',
  'to',
  'create',
  'a',
  'better',
  'result'],
 ['passion',
  'desire',
  'to',
  'provide',
  'long-term',
  'commitment',
  'to',
  'their',
  'organization',
  'demonstrate',
  'peak',
  'performance',
  'and',
  'maintain',
  'increased',
  'tenure',
  'with',
  'the',
  'organization',
  'with',
  'the',
  'intense',
  'desire',
  'or',
  'enthusiasm',
  'for',
  'the',
  'work'],
 ['teamwork',
  'ability',
  'to',
  'adapt',
  'to',
  'the',
  'needs',
  'of',
  'a',
  'group',
  'of',
  'people',
  'while',
  'supporting',
  'the',
  'work',
  'of',
  'others'],
 ['reliability',
  'ability',
  'to',
  'performing',
  'to',
  'perform',
  'when',
  'needed',
  'finishing',
  'projects',
  'and',
  'meeting',
  'deadlines'],
 ['people_skills',
  'ability',
  'to',
  'establish',
  'social',
  'connections',
  'and',
  'understand',
  'the',
  'mindset',
  'of',
  'others'],
 ['employee_loyal

In [57]:
model.build_vocab(phrase[comps * 5], update=True)
model.train(phrase[comps])

248

In [25]:
new_comp = """
Charisma	Able to establish social connections and a broad network
Confidence	Acts on his/her own initiative with confidence
Creativity	Seeks change and performs creatively
Entrepreneurial	Keeps aware of both organizational issues and market opportunities
Goal Orientated	Works systematically and drives the project to its goals
Goal Orientation	Work systematically and drive the project to the target goals
Growth Potential	Growth Potential
Integrity	Acts with integrity and shows social responsibility
Intellect	Flexible application of knowledge to differing contexts
Job Performance	Job Performance
Leadership	Identifies talents, empowers and motivates other
Logic	Logical thinking and rational analysis
Organisation	Sets objectives, balances resources and time, monitors progress
Organization	Sets objectives, balances resources and time, monitors progress
Performance	Job Performance
Proactive	Proactively deals with ambiguity
Process driven	Demonstrate commitment, accountablity, follow policies and procedures of the organizations
Resilient	Coping and resilient under stressful environment
Self motivated	Work enthusiastically for the achievement of self-development
Strategic Competency	Strategic Competency
Strategy	Foresight for potential problems and comes up with appropriate solutions
Teamwork	Adapts to the team; Supports others
"""

lines = new_comp.lower().strip().split('\n')
new_comp_dict = {splitted[0] : splitted[1] for splitted in [line.split('\t') for line in lines]}
for c in new_comp_dict:
    new_comp_dict[c] = new_comp_dict[c].replace(',', '')
new_comp_dict

{'charisma': 'able to establish social connections and a broad network',
 'confidence': 'acts on his/her own initiative with confidence',
 'creativity': 'seeks change and performs creatively',
 'entrepreneurial': 'keeps aware of both organizational issues and market opportunities',
 'goal orientated': 'works systematically and drives the project to its goals',
 'goal orientation': 'work systematically and drive the project to the target goals',
 'growth potential': 'growth potential',
 'integrity': 'acts with integrity and shows social responsibility',
 'intellect': 'flexible application of knowledge to differing contexts',
 'job performance': 'job performance',
 'leadership': 'identifies talents empowers and motivates other',
 'logic': 'logical thinking and rational analysis',
 'organisation': 'sets objectives balances resources and time monitors progress',
 'organization': 'sets objectives balances resources and time monitors progress',
 'performance': 'job performance',
 'proactive'

In [65]:
for nc in new_comp_dict:
    query = '_'.join(nc.lower().split())
    if query in model.vocab:
        sims = [[model.similarity(query, '_'.join(comp.split())), '_'.join(comp.split())] for comp in competency]
        sorted_sims = sorted(sims, key=lambda x: -x[0])
        print "{:25}\t{:20}\t{}".format(query, sorted_sims[0][1], sorted_sims[0][0])
    
    else:
        # print nc
        query = nc.lower().split()
        # print query
        sims = [[model.wmdistance(query, c.split()), c] for c in competency]
        # print sims
        sorted_sims = sorted(sims, key=lambda x: x[0])
        #print sorted_sims
        print "{:20}\t{:20}\t{}".format(nc, sorted_sims[0][1], sorted_sims[0][0])

strategic competency	teamwork            	8.14092422804
growth potential    	creativity          	16.7596804409
integrity                	reliability         	0.593463045389
intellect                	creativity          	0.648571967405
confidence               	leadership          	0.47871163808
goal orientated     	proactivity         	10.2819537665
job performance     	teamwork            	17.6008174822
organisation             	leadership          	0.428867019519
creativity               	creativity          	1.0
self motivated      	teamwork            	10.7826624786
strategy                 	leadership          	0.399676712378
charisma                 	creativity          	0.55684917215
teamwork                 	teamwork            	1.0
resilient                	proactivity         	0.208003836793
performance              	reliability         	0.521339674564
goal orientation    	teamwork            	15.3406456839
leadership               	leadership          	1.0
entrepreneurial  

In [44]:
query = 'charisma'.split()
print query
sims = [[model.wmdistance(query, c.split()), c] for c in competency]
# print sims
sorted_sims = sorted(sims, key=lambda x: x[0])
# print sorted_sims
print sorted_sims[0]

['charisma']
[4.844351291656494, 'teamwork']


In [12]:
s2 = 'leadership'.split()
s2

['leadership']

In [17]:
distance = model.wmdistance(s1, s2)

In [19]:
distance

19.93280601501465

In [15]:
model['liji']

KeyError: 'liji'